In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [11]:
# Caminho para a pasta com os resultados
input_path = '../input'

# Dicionário para armazenar os DataFrames nomeados
sheets = {}

# Lista para consolidar métricas
metrics_summary = []

residuais_por_cluster = {}

In [12]:
for filename in os.listdir(input_path):
    if filename.endswith('.csv'):
        model_name = filename.replace('.csv', '')
        model, cluster = model_name.split('_')
        cluster = int(cluster)

        file_path = os.path.join(input_path, filename)
        df = pd.read_csv(file_path, sep=r'\s+|,', engine='python')

        df['Residual_Error'] = df['Real_Requests'] - df['Predicted_Requests']
        df['Absolute_Error'] = df['Residual_Error'].abs()
        df['APE'] = np.where(df['Real_Requests'] != 0,
                             df['Absolute_Error'] / df['Real_Requests'],
                             np.nan)

        residual = df['Residual_Error']
        ape = df['APE'].dropna()

        summary = {
            'Cluster': cluster,
            'Modelo': model_name,
            'MAPE (%)': round(ape.mean() * 100, 2),
            'Q2 (Mediana)': round(residual.median(), 2),
            'Q1': round(residual.quantile(0.25), 2),
            'Q3': round(residual.quantile(0.75), 2),
            'Erro Mínimo': round(residual.min(), 2),
            'Erro Máximo': round(residual.max(), 2),
            'Média': round(residual.mean(), 2),
            'Desvio Padrão': round(residual.std(), 2)
        }

        metrics_summary.append(summary)
        sheets[model_name] = df

# Criação da tabela consolidada e ordenação
metrics_df = pd.DataFrame(metrics_summary)
metrics_df.sort_values(by=['Cluster', 'Modelo'], inplace=True)

In [13]:
metrics_df

,Cluster,Modelo,MAPE (%),Q2 (Mediana),Q1,Q3,Erro Mínimo,Erro Máximo,Média,Desvio Padrão
23,0,BLR_0,3.01,-6.0,-92.00,87.00,-5953,6034,-0.14,280.16
14,0,EWA_0,3.93,-6.5,-127.00,117.00,-5896,6126,-0.50,340.70
7,0,KNN_0,6.14,0.0,-124.00,127.00,-7845,6590,16.34,683.05
15,0,SKL_0,4.61,-13.5,-141.25,144.25,-6014,6157,1.17,362.14
24,0,SRP_0,5.16,-2.0,-144.25,141.00,-6128,6162,-16.59,380.53
5,1,BLR_1,2.48,-7.0,-104.00,101.00,-6081,5797,0.51,270.62
16,1,EWA_1,3.35,-6.0,-142.00,137.00,-6293,5858,-0.53,336.56
6,1,KNN_1,5.43,2.0,-148.00,151.00,-5891,5832,19.13,654.48
18,1,SKL_1,3.90,-23.0,-172.00,161.00,-6150,5937,-1.17,375.36
1,1,SRP_1,4.12,5.0,-151.00,179.25,-6449,5918,6.15,396.51


In [14]:
# Agrupar os erros por cluster
for filename in os.listdir(input_path):
    if filename.endswith('.csv'):
        model_name = filename.replace('.csv', '')
        model, cluster = model_name.split('_')
        cluster = int(cluster)

        file_path = os.path.join(input_path, filename)
        df = pd.read_csv(file_path, sep=r'\s+|,', engine='python')

        df['Residual_Error'] = df['Real_Requests'] - df['Predicted_Requests']

        if cluster not in residuais_por_cluster:
            residuais_por_cluster[cluster] = {}

        residuais_por_cluster[cluster][model_name] = df['Residual_Error'].values

In [15]:
# Dicionário de cores fixas por modelo
cores_modelos = {
    'KNN': 'blue',
    'EWA': 'orange',
    'SKL': 'green',
    'BLR': 'red',
    'SRP': 'purple'
}

In [16]:
# Gerar os gráficos de CDF por cluster
pdf_file = 'cdf_erros.pdf'


with PdfPages(pdf_file) as pdf:
    for cluster, modelos in sorted(residuais_por_cluster.items()):
        plt.figure(figsize=(8, 6))
        for modelo, erros in modelos.items():
            sorted_errors = np.sort(erros)
            cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
            plt.plot(sorted_errors, cdf, label=modelo)

        plt.title(f'CDF dos Erros - Cluster {cluster}')
        plt.xlabel('Erro Residual')
        plt.ylabel('Frequência Acumulada')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        pdf.savefig()  # Salva a figura atual no PDF
        plt.close()

print(f'Gráficos salvos em {pdf_file}')

Gráficos salvos em cdf_erros.pdf


In [17]:
# Gerar os gráficos de CDF por cluster com escala log no eixo x
pdf_file = 'cdf_erros_log.pdf'

with PdfPages(pdf_file) as pdf:
    for cluster, modelos in sorted(residuais_por_cluster.items()):
        plt.figure(figsize=(8, 6))
        for modelo, erros in modelos.items():
            sorted_errors = np.sort(erros)
            cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
            plt.plot(sorted_errors, cdf, label=modelo)

        plt.title(f'CDF dos Erros - Cluster {cluster}')
        plt.xlabel('Erro Residual (escala log)')
        plt.ylabel('Frequência Acumulada')
        plt.xscale('log')  # Escala logarítmica no eixo x
        plt.grid(True, linestyle='--', alpha=0.7, which='both')
        plt.legend()
        pdf.savefig()
        plt.close()

print(f'Gráficos salvos em {pdf_file}')

Gráficos salvos em cdf_erros_log.pdf


In [18]:
pdf_file = 'cdf_erros_logy.pdf'

with PdfPages(pdf_file) as pdf:
    for cluster, modelos in sorted(residuais_por_cluster.items()):
        plt.figure(figsize=(8, 6))
        for modelo, erros in modelos.items():
            # Pega apenas o prefixo do nome (antes do "_") para associar a cor
            nome_base = modelo.split('_')[0]
            cor = cores_modelos.get(nome_base, None)

            sorted_errors = np.sort(erros)
            cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
            cdf = np.clip(cdf, 1e-6, 1)  # evita problemas com log(0)

            plt.plot(sorted_errors, cdf, label=modelo, color=cor)

        plt.title(f'CDF dos Erros - Cluster {cluster}')
        plt.xlabel('Erro Residual')
        plt.ylabel('Frequência Acumulada (escala log)')
        plt.yscale('log')
        plt.grid(True, linestyle='--', alpha=0.7, which='both')
        plt.legend()
        pdf.savefig()
        plt.close()

print(f'Gráficos salvos em {pdf_file}')

Gráficos salvos em cdf_erros_logy.pdf


In [22]:
# New PDF with vertically stacked plots for clusters 0 and 4
pdf_file_duplo = 'cdf_errors_logy_cluster0_4.pdf'

with PdfPages(pdf_file_duplo) as pdf:
    fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True)

    for idx, cluster in enumerate([0, 4]):
        ax = axs[idx]
        modelos = residuais_por_cluster[cluster]

        for modelo, erros in modelos.items():
            nome_base = modelo.split('_')[0]
            cor = cores_modelos.get(nome_base, None)

            sorted_errors = np.sort(erros)
            cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
            cdf = np.clip(cdf, 1e-6, 1)

            ax.plot(sorted_errors, cdf, label=modelo, color=cor)

        ax.set_title(f'Error CDF – Cluster {cluster}')
        ax.set_ylabel('Cumulative Frequency (log scale)')
        ax.set_yscale('log')
        ax.grid(True, linestyle='--', alpha=0.7, which='both')
        ax.legend()

    axs[1].set_xlabel('Residual Error')
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close()

print(f'Stacked plot saved to {pdf_file_duplo}')


Stacked plot saved to cdf_errors_logy_cluster0_4.pdf


In [28]:
# Verificando os valores mais negativos do modelo BLR no cluster 4
modelo = 'SRP_4'
cluster = 4

# Recupera os erros residuais
erros_blr_cluster4 = residuais_por_cluster[cluster][modelo]

# Ordena do menor para o maior
erros_ordenados = np.sort(erros_blr_cluster4)

# Exibe os 10 valores mais negativos
print("10 erros mais negativos do SRP no cluster 4:")
print(erros_ordenados[:10])

10 erros mais negativos do SRP no cluster 4:
[-40647 -12986  -9334  -8677  -6519  -6459  -6448  -6411  -6409  -5831]


In [21]:
# Salvando no arquivo Excel
output_file = 'analises_modelos.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
    metrics_df.to_excel(writer, sheet_name='Resumo_Metricas', index=False)

print(f'Relatório salvo como {output_file}')

Relatório salvo como analises_modelos.xlsx


2	4	BLR_4	48.72	-122.0	-349.50	280.00	-34842	37419	3.84	1258.48
4	4	EWA_4	54.00	-47.0	-639.50	512.50	-32165	37021	-6.63	1565.29
22	4	KNN_4	54.75	-22.0	-581.00	442.00	-26503	38181	60.97	3104.76
17	4	SKL_4	79.46	-122.0	-780.00	553.00	-14823	37834	16.46	1831.42
20	4	SRP_4	51.49	3.0	-596.00	627.00	-40647	37393	59.44	1846.25
